# Mid-term Project

In [622]:
# Importing necessary libraries
import os
import pandas as pd
import numpy as np
from pymongo import MongoClient
import json
import datetime
import mysql.connector
from sqlalchemy import create_engine, text
import sqlalchemy
import pymongo
import certifi


In [624]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.30
Running PyMongo Version: 4.10.1


In [626]:
mysql_args = {
    "uid" : "root",
    "pwd" : "unicornTree98*!",
    "hostname" : "localhost",
    "dbname" : "sakila"
}

src_dbname = "sakila"
dst_dbname = "sakila_dw"

mongodb_args = {
    "user_name" : "paige7",
    "password" : "OxzQQK8aGjaUHcgT",
    "cluster_name" : "Cluster0",
    "cluster_subnet" : "vl73d",
    "cluster_location" : "atlas", # "local"
    "db_name" : "sakila"}

In [628]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()
    


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [630]:
conn_str = f"mysql+pymysql://{mysql_args['uid']}:{mysql_args['pwd']}@{mysql_args['hostname']}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()

In [632]:
sakila_staff = "SELECT * FROM sakila.staff;"
df_staff = get_sql_dataframe(sakila_staff, **mysql_args)
df_staff.head(2)

,staff_id,first_name,last_name,address_id,picture,email,store_id,active,username,password,last_update
0,1,Mike,Hillyer,3,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Mike.Hillyer@sakilastaff.com,1,1,Mike,8cb2237d0679ca88db6464eac60da96345513964,2006-02-15 03:57:16
1,2,Jon,Stephens,4,None,Jon.Stephens@sakilastaff.com,2,1,Jon,None,2006-02-15 03:57:16


In [634]:
sakila_film = "SELECT * FROM sakila.film;"
df_film = get_sql_dataframe(sakila_film, **mysql_args)
df_film.head(2)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42


In [636]:
sakila_store = "SELECT * FROM sakila.store;"
df_store = get_sql_dataframe(sakila_store, **mysql_args)
df_store.head(2)

,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2006-02-15 04:57:12
1,2,2,2,2006-02-15 04:57:12


In [638]:
sakila_customer = "SELECT * FROM sakila.customer;"
df_customer = get_sql_dataframe(sakila_customers, **mysql_args)
df_customer.head(2)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [640]:
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook
data_dir = os.path.join(os.getcwd(), 'Downloads')

json_files = {"inventory" : 'sakila-inventory.json'}

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)  

In [641]:
# Extracting Film Data from MongoDb collection
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "inventory"

# Here, we are extracting the film collection from the MongoDB database
df_inventory = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_inventory.head()

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17


In [642]:
data_dir = os.path.join(os.getcwd(), 'downloads')
data_file = os.path.join(data_dir, 'sakila-payment.csv')
df_payment = pd.read_csv(data_file, sep=";", header=0, index_col=0)
df_payment.head()

,customer_id,staff_id,rental_id,amount,payment_date,last_update
payment_id,,,,,,
1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30
3,1,1,1185,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30
4,1,2,1422,0.99,2005-06-15 18:02:53,2006-02-15 22:12:30
5,1,2,1476,9.99,2005-06-15 21:08:46,2006-02-15 22:12:30


In [643]:
# Transformations

In [644]:
drop_cols = ['address_id','picture','password', 'last_update', 'active', 'store_id']
df_staff.drop(drop_cols, axis=1, inplace=True)
df_staff.rename(columns={"staff_id":"staff_key"}, inplace=True)

df_staff.head(2)

,staff_key,first_name,last_name,email,username
0,1,Mike,Hillyer,Mike.Hillyer@sakilastaff.com,Mike
1,2,Jon,Stephens,Jon.Stephens@sakilastaff.com,Jon


In [645]:
drop_cols = ['language_id','original_language_id', 'last_update']
df_film.drop(drop_cols, axis=1, inplace=True)
df_film.rename(columns={"film_id":"film_key"}, inplace=True)
df_inventory.rename(columns={"film_id": "film_key"}, inplace=True)
df_film.head(2)

,film_key,title,description,release_year,rental_duration,rental_rate,length,replacement_cost,rating,special_features
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,3,4.99,48,12.99,G,"Trailers,Deleted Scenes"


In [646]:
drop_cols = ['manager_staff_id','last_update']
df_store.drop(drop_cols, axis=1, inplace=True)
df_store.rename(columns={"store_id":"store_key"}, inplace=True)
df_store.head(2)

df_inventory.rename(columns={"store_id": "store_key"}, inplace=True)


In [647]:
drop_cols = ['address_id','active','create_date','last_update']
df_customer.drop(drop_cols, axis=1, inplace=True)
df_customer.rename(columns={"customer_id":"customer_key"}, inplace=True)
df_customer.head(2)

df_customer.rename(columns={"customer_id": "customer_key"}, inplace=True)

In [648]:
drop_cols = ['last_update']
df_inventory.drop(drop_cols, axis=1, inplace=True)
df_inventory.rename(columns={"inventory_id":"inventory_key"}, inplace=True)

df_inventory.head(2)

,inventory_key,film_key,store_key
0,1,1,1
1,2,1,1


In [649]:
drop_cols = ['last_update']
df_payment.drop(drop_cols, axis=1, inplace=True)
df_payment.rename(columns={"payment_id":"payment_key"}, inplace=True)
df_payment.head(2)

,customer_id,staff_id,rental_id,amount,payment_date
payment_id,,,,,
1,1,1,76,2.99,2005-05-25 11:30:37
2,1,1,573,0.99,2005-05-28 10:35:23


In [660]:
# Merging Film, Customer, and Inventory into Fact Rentals

In [662]:
df_fact_inventory1 = pd.merge(df_inventory, df_film, on='film_key', how='inner')
df_fact_inventory1.rename(columns={"title":"movie_title", "customer_id":"customer_key","staff_id":"staff_key"}, inplace=True)
df_fact_inventory1.head(2)

,inventory_key,film_key,store_key,movie_title,description,release_year,rental_duration,rental_rate,length,replacement_cost,rating,special_features
0,1,1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
1,2,1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"


In [664]:
df_fact_inventory = pd.merge(df_fact_inventory1, df_store, on='store_key', how='inner')
df_fact_inventory.rename(columns={"active":"active_customer"}, inplace=True)
df_fact_inventory.head(2).sort_values(by=['inventory_key'])

,inventory_key,film_key,store_key,movie_title,description,release_year,rental_duration,rental_rate,length,replacement_cost,rating,special_features,address_id
0,1,1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",1
1,2,1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",1


In [666]:
# Date Dimension

In [680]:
#Create Date sql_dim_date = "SELECT date_key, full_date FROM sakila_dw.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(5)

,date_key,full_date
0,20200101,2020-01-01
1,20200102,2020-01-02
2,20200103,2020-01-03
3,20200104,2020-01-04
4,20200105,2020-01-05


In [682]:
# Extract Customer Dimension
sql_customers = "SELECT * FROM sakila.customer"
df_customers = get_sql_dataframe(sql_customers, **mysql_args)

# Map create_date and last_update to dim_date surrogate keys
df_dim_create_date = df_dim_date.rename(columns={"date_key": "create_date_key", "full_date": "create_date"})
df_customers.create_date = pd.to_datetime(df_customers.create_date).dt.date
df_customers = pd.merge(df_customers, df_dim_create_date, on='create_date', how='left').drop(columns=["create_date"])

df_dim_last_update = df_dim_date.rename(columns={"date_key": "last_update_key", "full_date": "last_update"})
df_customers.last_update = pd.to_datetime(df_customers.last_update).dt.date
df_customers = pd.merge(df_customers, df_dim_last_update, on='last_update', how='left').drop(columns=["last_update"])
df_customers.head(5)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date_key,last_update_key
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,NaN,NaN
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,NaN,NaN
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,NaN,NaN
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,NaN,NaN
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,NaN,NaN
